In [ ]:
from framed import load_cbmodel, pFBA, FBA, Community, Environment
from framed.experimental.visualization import build_escher_map

## Exercise: simple community with two *E. coli* mutants

First, let's load a simple model of central carbon metabolism and make a copy of the model

In [ ]:
model1 = load_cbmodel('models/e_coli_core.xml', flavor='bigg')
model1.reactions.R_ATPM.lb = 0 # we will ignore maintenance ATP for this exercise 
model2 = model1.copy()

The first model is a deletion mutant for ammonium uptake:

In [ ]:
model1.id = 'mutA'
model1.set_flux_bounds('R_NH4t', 0, 0)

The second model is a deletion mutant for glucose uptake: 

In [ ]:
model2.id = 'mutB'
model2.set_flux_bounds('R_GLCpts', 0, 0)

What happens if we try to grow each model individually in minimal medium (pre-defined environment) ? 

In [ ]:
print FBA(model1)
print FBA(model2)

Let's clear the individual environmental constraints (we will apply the constraints directly at the community level).

In [ ]:
env = Environment.from_model(model1)
Environment.complete(model1, inplace=True)
Environment.complete(model2, inplace=True)

In [ ]:
community = Community('test', [model1, model2], merge_extracellular_compartments=True,
                      extracellular_compartment_id='e')

In [ ]:
env.apply(community.merged, inplace=True)
print 'Compounds in the minimal medium:', ', '.join(env.get_compounds())

Now let's run a (p)FBA simulation for the whole community:

In [ ]:
sol = pFBA(community.merged)
org_fluxes = community.split_fluxes(sol.values)

In [ ]:
for org, biomass in community.organisms_biomass_reactions.items():
    print 'Growth', org, sol.values[biomass]

Let's check what each mutant is doing:

In [ ]:
build_escher_map(org_fluxes['mutA'], 'e_coli_core.Core metabolism').display_in_notebook()

In [ ]:
build_escher_map(org_fluxes['mutB'], 'e_coli_core.Core metabolism').display_in_notebook()